In [1]:
# Step 1 --> Installing required packages
!pip install opencv-python numpy matplotlib torch torchvision ultralytics

In [2]:
# Step 2 --> Importing important libraries 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
from collections import defaultdict


In [3]:
# Step 3 --> Loading pretrained YOLOv8 model 
model = YOLO("yolov8n.pt") 


In [4]:
# Step 4 --> Defining Vehicle Classes
# Class IDs in COCO for vehicles
TARGET_CLASSES = {'car': 2, 'motorcycle': 3, 'truck': 7}

# Reverse class name mapping
CLASS_NAMES = {2: 'car', 3: 'motorcycle', 7: 'truck'}


In [5]:
#Step 5 --> Defining Image Processing Function
def process_image(image_path, conf_threshold=0.5):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = model(img_rgb)[0]
    
    vehicle_counts = defaultdict(int)

    for box in results.boxes:
        cls_id = int(box.cls.item())
        conf = float(box.conf.item())
        if conf >= conf_threshold and cls_id in CLASS_NAMES:
            label = CLASS_NAMES[cls_id]
            vehicle_counts[label] += 1
            # Draw box
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
            text = f"{label} {conf:.2f}"
            cv2.putText(img, text, (xyxy[0], xyxy[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    return img, vehicle_counts


In [6]:
# Step 6 --> Batch Processing test images
input_folder = r"C:\Users\khush\Downloads\traffic_dataset\input data"
output_folder = r"C:\Users\khush\Downloads\traffic_dataset\output data\processed_image"
os.makedirs(output_folder, exist_ok=True)

summary_report = []

for image_name in os.listdir(input_folder):
    if image_name.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(input_folder, image_name)
        processed_img, counts = process_image(image_path)
        
        # Save annotated image
        output_path = os.path.join(output_folder, f"annotated_{image_name}")
        cv2.imwrite(output_path, processed_img)

        summary_report.append({'image': image_name, 'counts': dict(counts)})



0: 384x640 3 persons, 2 buss, 1 truck, 1 umbrella, 478.6ms
Speed: 32.2ms preprocess, 478.6ms inference, 17.9ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 3 cars, 1 traffic light, 555.6ms
Speed: 9.7ms preprocess, 555.6ms inference, 5.5ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 3 persons, 1 truck, 266.4ms
Speed: 4.0ms preprocess, 266.4ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 person, 1 bicycle, 1 traffic light, 319.1ms
Speed: 5.4ms preprocess, 319.1ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 5 persons, 2 cars, 2 trucks, 2 backpacks, 1 handbag, 371.9ms
Speed: 3.7ms preprocess, 371.9ms inference, 2.7ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 person, 1 truck, 365.2ms
Speed: 4.2ms preprocess, 365.2ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 448)

0: 448x640 1 motorcycle, 1 truck, 308.8ms
Speed: 5.1ms preprocess, 308.8ms inference, 4.5ms po

In [ ]:
# Visualizing Processed Image
img_show = cv2.imread(output_path)
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(img_show, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Annotated Output")
plt.show()


In [ ]:
# Step 8 --> Generating Summary Report
import pandas as pd

summary_df = pd.DataFrame(summary_report)
summary_df['Total'] = summary_df['counts'].apply(lambda d: sum(d.values()))
summary_df
